In [1]:
import numpy as np
import copy 
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
# Main C3 objects
from c3.c3objs import Quantity as Qty
from c3.parametermap import ParameterMap as PMap
from c3.experiment import Experiment as Exp
from c3.model import Model as Mdl
from c3.generator.generator import Generator as Gnr

# Building blocks
import c3.generator.devices as devices
import c3.signal.gates as gates
import c3.libraries.chip as chip
import c3.signal.pulse as pulse
import c3.libraries.tasks as tasks

# Libs and helpers
import c3.libraries.algorithms as algorithms
import c3.libraries.hamiltonians as hamiltonians
import c3.libraries.fidelities as fidelities
import c3.libraries.envelopes as envelopes
import c3.utils.qt_utils as qt_utils
import c3.utils.tf_utils as tf_utils

#%matplotlib widget

import plotly.express as px
import plotly.graph_objects as go

2021-10-25 12:24:04.621649: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-25 12:24:04.621703: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
Num_qubits = 5
Num_coupler = Num_qubits - 1

qubit_levels = 3
coupler_levels = 3

freq_q_arr = [4e9, 4e9, 4e9, 4e9, 4e9]
anharm_q_arr = [-210e6, -210e6, -210e6, -210e6, -210e6]
t1_q_arr = [27e-6, 27e-6, 27e-6, 27e-6, 27e-6]
t2star_q_arr = [39e-6, 39e-6, 39e-6, 39e-6, 39e-6]

freq_c_arr = [4e9, 4e9, 4e9, 4e9]
anharm_c_arr = [-210e6, -210e6, -210e6, -210e6]
t1_c_arr = [27e-6, 27e-6, 27e-6, 27e-6]
t2star_c_arr = [39e-6, 39e-6, 39e-6, 39e-6]

qubit_temp = 50e-3
phi = 0.1
phi_0 = 10.0
d = 0.0

In [4]:
qubit_array = []
coupler_array = []

for i in range(Num_qubits):
        qubit_array.append( 
                chip.Transmon(
                        name = "Q" + str(i+1),
                        desc = "Qubit  "+ str(i+1),
                        freq = Qty(
                                value = freq_q_arr[i],
                                min_val = 3.995e9,
                                max_val = 4.005e9,
                                unit = "Hz 2pi"
                        ),
                        anhar = Qty(
                                value = anharm_q_arr[i],
                                min_val = -380e6,
                                max_val = -120e6,
                                unit = "Hz 2pi"
                        ),
                        hilbert_dim = qubit_levels,
                        t1 = Qty(
                                value = t1_q_arr[i],
                                min_val = 1e-6,
                                max_val = 90e-6,
                                unit = "s"
                        ),
                        t2star=Qty(
                                value=t2star_q_arr[i],
                                min_val=10e-6,
                                max_val=90e-6,
                                unit="s"
                        ),
                        temp=Qty(
                                value=qubit_temp,
                                min_val=0.0,
                                max_val=0.12,
                                unit="K"
                        ),
                        phi=Qty(
                                value=phi,
                                max_val=5.0,
                                min_val=0.0,
                                unit = "Wb"
                        ),
                        phi_0 = Qty(
                                value = phi_0,
                                max_val = 11.0,
                                min_val = 9.0,
                                unit = "Wb"
                        ),
                        d = Qty(
                                value=d,
                                max_val=0.1,
                                min_val=-0.1,
                                unit = ""
                        )
                        
                )
        )


for i in range(Num_coupler):
        coupler_array.append( 
                chip.Transmon(
                        name = "C" + str(i+1),
                        desc = "Coupler  "+ str(i+1),
                        freq = Qty(
                                value = freq_c_arr[i],
                                min_val = 3.995e9,
                                max_val = 4.005e9,
                                unit = "Hz 2pi"
                        ),
                        anhar = Qty(
                                value = anharm_c_arr[i],
                                min_val = -380e6,
                                max_val = -120e6,
                                unit = "Hz 2pi"
                        ),
                        hilbert_dim = coupler_levels,
                        t1 = Qty(
                                value = t1_c_arr[i],
                                min_val = 1e-6,
                                max_val = 90e-6,
                                unit = "s"
                        ),
                        t2star=Qty(
                                value=t2star_c_arr[i],
                                min_val=10e-6,
                                max_val=90e-6,
                                unit="s"
                        ),
                        temp=Qty(
                                value=qubit_temp,
                                min_val=0.0,
                                max_val=0.12,
                                unit="K"
                        ),
                        phi=Qty(
                                value=phi,
                                max_val=5.0,
                                min_val=0.0,
                                unit = "Wb"
                        ),
                        phi_0 = Qty(
                                value = phi_0,
                                max_val = 11.0,
                                min_val = 9.0,
                                unit = "Wb"
                        ),
                        d = Qty(
                                value=d,
                                max_val=0.1,
                                min_val=-0.1,
                                unit = ""
                        )
                        
                )
        )

In [5]:
NN_coupling_strength_values = [20e6, 20e6, 20e6, 20e6, 20e6, 20e6, 20e6, 20e6, 20e6]
g_NN_array = []


for i in range(Num_coupler):
    for j in range(Num_qubits):
        if j == i + 1 or j == i: 
            g_NN_array.append(
                chip.Coupling(
                    name=qubit_array[j].name + "-" + coupler_array[i].name,
                    desc="Coupling",
                    comment="Coupling between "+ qubit_array[j].name + " and " + coupler_array[i].name,
                    connected=[qubit_array[j].name, coupler_array[i].name],
                    strength=Qty(
                            value=NN_coupling_strength_values[i],
                            min_val=-1*1e3,
                            max_val=200e6,
                            unit="Hz 2pi"
                    ),
                    hamiltonian_func=hamiltonians.int_XX
                )
            )
        else:
            continue


for i in g_NN_array:
    print(i.connected)


['Q1', 'C1']
['Q2', 'C1']
['Q2', 'C2']
['Q3', 'C2']
['Q3', 'C3']
['Q4', 'C3']
['Q4', 'C4']
['Q5', 'C4']


In [6]:
# Drives for qubits

drive_array = []

for i in range(Num_qubits):
    drive_array.append(
        chip.Drive(
        name="d" + str(i+1),
        desc="Drive " + str(i+1),
        comment = "Drive line" +str(i+1) +  "on qubit "+str(i+1),
        connected = [qubit_array[i].name],
        hamiltonian_func = hamiltonians.x_drive
        )
    )


for i in drive_array:
    print(i.connected)

['Q1']
['Q2']
['Q3']
['Q4']
['Q5']


In [7]:
m00_arr = [0.97, 0.97, 0.97, 0.97, 0.97] 
m01_arr = [0.04, 0.04, 0.04, 0.04, 0.04]

one_zeros = np.array([0] * qubit_levels)
zero_ones = np.array([1] * qubit_levels)
one_zeros[0] = 1
zero_ones[0] = 0

confusion_row_arr = []
for i in range(Num_qubits):
    val = one_zeros * m00_arr[i] + zero_ones * m01_arr[i]

    min_val = one_zeros * 0.8 + zero_ones * 0.0
    max_val = one_zeros * 1.0 + zero_ones * 0.2

    confusion_row_arr.append( Qty(value=val, min_val=min_val, max_val=max_val, unit="") )

conf_matrix = tasks.ConfusionMatrix(Q1=confusion_row_arr[0], 
                                    Q2=confusion_row_arr[1],
                                    Q3=confusion_row_arr[2],
                                    Q4=confusion_row_arr[3],
                                    Q5=confusion_row_arr[4])


In [8]:

init_temp = 50e-3
init_ground = tasks.InitialiseGround(
        init_temp = Qty(
                value = init_temp,
                min_val = -0.001,
                max_val = 0.22,
                unit = "K"
        )
)

In [9]:
model = Mdl(
        qubit_array + coupler_array,
        drive_array + g_NN_array,
        [conf_matrix, init_ground]
)
model.set_lindbladian(False)
model.set_dressed(True)

In [ ]:
sim_res = 100e9
awg_res = 2e9
lo = devices.LO(name="lo", resolution=sim_res)
awg = devices.AWG(name = "awg", resolution=awg_res)
mixer = devices.Mixer(name = "mixer")

resp = devices.Response(
        name = "resp",
        rise_time = Qty(
                value = 0.3e-9,
                min_val = 0.05e-9,
                max_val = 0.6e-9,
                unit = "s"
        ),
        resolution = sim_res

)

dig_to_an = devices.DigitalToAnalog(
                name = "dac",
                resolution = sim_res
)

v2hz = 1e9

v_to_hz = devices.VoltsToHertz(
                name = "V_to_Hz",
                V_to_Hz = Qty(
                        value = v2hz,
                        min_val = 0.9e9,
                        max_val = 1.1e9,
                        unit = "Hz/V"
                )
)

generator = Gnr(
        devices = {
                "LO": devices.LO(name="lo", resolution = sim_res, outputs= 1),
                "AWG": devices.AWG(name = "awg", resolution = awg_res, outputs = 1),
                "DigitalToAnalog": devices.DigitalToAnalog(
                        name = "dac",
                        resolution = sim_res,
                        inputs = 1,
                        outputs = 1
                ),
                "Response": devices.Response(
                        name = "resp",
                        rise_time = Qty(
                                value = 0.3e-9,
                                min_val = 0.05e-9,
                                max_val = 0.6e-9,
                                unit = "s"
                        ),
                        resolution = sim_res,
                        inputs = 1,
                        outputs = 1
                ),
                "Mixer": devices.Mixer(name = "mixer", inputs = 2, outputs = 1),
                "VoltsToHertz": devices.VoltsToHertz(
                        name = "V_to_Hz",
                        V_to_Hz = Qty(
                                value = v2hz,
                                min_val = 0.9e9,
                                max_val = 1.1e9,
                                unit = "Hz/V"
                        ),
                        inputs= 1,
                        outputs = 1
                )
        },
        
        chains = {
                "d1": ["LO", "AWG", "DigitalToAnalog", "Response", "Mixer", "VoltsToHertz"],
                "d2": ["LO", "AWG", "DigitalToAnalog", "Response", "Mixer", "VoltsToHertz"],
                "d3": ["LO", "AWG", "DigitalToAnalog", "Response", "Mixer", "VoltsToHertz"],
                "d4": ["LO", "AWG", "DigitalToAnalog", "Response", "Mixer", "VoltsToHertz"],
                "d5": ["LO", "AWG", "DigitalToAnalog", "Response", "Mixer", "VoltsToHertz"]
        }

)

t_final = 7e-9
sideband = 50e6
gauss_params_single = {
        "amp": Qty(
                value = 0.5,
                min_val = 0.2,
                max_val = 0.6,
                unit = "V"
        ),
        "t_final": Qty(
                value = t_final,
                min_val = 0.5 * t_final,
                max_val = 1.5 * t_final,
                unit = "s"
        ),
        "sigma": Qty(
                value = t_final/4,
                min_val = t_final/8,
                max_val = t_final/2,
                unit = "s"
        ),
        "xy_angle": Qty(
                value = 0.0,
                min_val = -0.5 * np.pi,
                max_val = 2.5 * np.pi,
                unit = "rad"
        ),
        "freq_offset": Qty(
                value = -sideband - 3e6,
                min_val = -56 * 1e6,
                max_val = -52 * 1e6,
                unit = "Hz 2pi"
        ),
        "delta": Qty(
                value = -1,
                min_val = -5,
                max_val = 3,
                unit = ""
        )
}


gauss_env_single = pulse.Envelope(
        name = "gauss",
        desc = "Gaussian comp for single qubit gates",
        params = gauss_params_single,
        shape = envelopes.gaussian_nonorm
)

nodrive_env = pulse.Envelope(
        name = "no_drive",
        params = {
                "t_final": Qty(
                        value = t_final,
                        min_val = 0.5 * t_final,
                        max_val = 1.5 * t_final,
                        unit = "s"
                )
        },
        shape = envelopes.no_drive
)

In [ ]:
qubit_freqs = model.get_qubit_freqs()

lo_freq_array = [qubit_freqs[i] + sideband for i in range(Num_qubits) ]

carrier_array = []

for i in range(Num_qubits):

        carrier_parameters = {
                "freq": Qty(
                        value = lo_freq_array[i],
                        min_val = 1e9,
                        max_val = 8e9,
                        unit = "Hz 2pi"
                ),
                "framechange": Qty(
                        value = 0.0,
                        min_val = -np.pi,
                        max_val = 3 * np.pi,
                        unit = "rad"
                )
        }

        carrier_array.append( 
                pulse.Carrier(
                        name = "carrier",
                        desc = "Frequency of the local oscillator",
                        params = carrier_parameters
                )
        )

In [ ]:
# Single qubit gates
rx90p_gate_array = []

for i in range(Num_qubits):
    rx90p_q = gates.Instruction(
            name = "rx90p", targets = [i], t_start = 0.0, t_end = t_final, channels=[drive_array[i].name,drive_array[(i+1)%Num_qubits].name]
    )
    rx90p_q.add_component(gauss_env_single, drive_array[i].name)
    rx90p_q.add_component(carrier_array[i], drive_array[i].name)


